# Test Metrics

In this Notebook the metrcis for the differen TN models are calculated

### imports

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import distance

In [ ]:
# load df
results = pd.read_csv('../results/Pre_Results_TN.csv')
results

,version,test_loss,accuracy,precision,recall,f1,time,fps,frame_results
0,"{'augmentation': True, 'input_number': 3, 'out...",4753.948033,0.872008,0.942683,0.901578,0.921673,8.694051,330.915950,"[{'subset': 'New', 'video': 'Video_1', 'clip':..."
1,"{'augmentation': False, 'input_number': 3, 'ou...",4161.026125,0.851197,0.923313,0.892418,0.907603,8.861190,324.674231,"[{'subset': 'New', 'video': 'Video_1', 'clip':..."
2,"{'augmentation': True, 'input_number': 3, 'out...",1795.755328,0.861661,0.946661,0.883312,0.913890,21.285650,135.161485,"[{'subset': 'New', 'video': 'Video_1', 'clip':..."
3,"{'augmentation': False, 'input_number': 3, 'ou...",2292.572355,0.808481,0.901713,0.857265,0.878928,21.145362,136.058206,"[{'subset': 'New', 'video': 'Video_1', 'clip':..."
4,"{'augmentation': True, 'input_number': 5, 'out...",4217.184523,0.829344,0.905800,0.882703,0.894102,9.226051,311.834388,"[{'subset': 'New', 'video': 'Video_1', 'clip':..."
5,"{'augmentation': False, 'input_number': 5, 'ou...",4885.998479,0.865765,0.946089,0.889859,0.917113,9.441132,304.730405,"[{'subset': 'New', 'video': 'Video_1', 'clip':..."
6,"{'augmentation': True, 'input_number': 5, 'out...",8495.575709,0.880690,0.957400,0.897489,0.926477,6.421600,448.019195,"[{'subset': 'New', 'video': 'Video_1', 'clip':..."
7,"{'augmentation': False, 'input_number': 5, 'ou...",6633.399764,0.862414,0.928326,0.903132,0.915556,6.857523,419.539229,"[{'subset': 'New', 'video': 'Video_1', 'clip':..."


### functions

In [3]:
# calculae metrics for row 
def calculate_metrics(row, threshold=4):
    predicted_position = row['predicted_position']
    actual_position = row['ground_truth_position']

    if actual_position == (-1,-1) and predicted_position == (-1,-1):
        # true negative
        return 0, 0, 1, 0
    elif actual_position == (-1,-1) and predicted_position != (-1,-1):
        # false positive
        return 0, 1, 0, 0
    elif actual_position != (-1,-1) and predicted_position == (-1,-1):
        # false negative
        return 0, 0, 0, 1
    else:
        if distance.euclidean(predicted_position, actual_position) < threshold:
            # true positive
            return 1, 0, 0, 0
        else:
            # false positive
            return 0, 1, 0, 0

In [4]:
# function to calculate complete metrics
def calculate_df_metrcis(frame_results_df, threshold=4):
    metrics_dict = {'Dataset_New': {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0},
                    'Dataset_TrackNet': {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0},
                    'Visibility_Visible': {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0},
                    'Visibility_Difficult': {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0},
                    'Visibility_Blurry': {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0},
                    'Visibility_Occluded': {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0},
                    'Visibility_None': {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0},
                    'Video_6': {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0},
                    'Total': {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0}}
    
    for i in range(len(frame_results_df)):
        row = frame_results_df.iloc[i]

        subset = "Dataset_" + row['subset']
        visibility = "Visibility_" + (row['points'][0]['visibility'] if  len(row['points']) > 0 else 'None')
        video = row['video']

        tp, fp, tn, fn = calculate_metrics(row, threshold)

        # subset metrics
        metrics_dict[subset]['tp'] += tp
        metrics_dict[subset]['fp'] += fp
        metrics_dict[subset]['tn'] += tn
        metrics_dict[subset]['fn'] += fn

        # visibility metrics
        metrics_dict[visibility]['tp'] += tp
        metrics_dict[visibility]['fp'] += fp
        metrics_dict[visibility]['tn'] += tn
        metrics_dict[visibility]['fn'] += fn

        #check if video is Video_1
        if video == 'Video_6':
            metrics_dict['Video_6']['tp'] += tp
            metrics_dict['Video_6']['fp'] += fp
            metrics_dict['Video_6']['tn'] += tn
            metrics_dict['Video_6']['fn'] += fn

        # total metrics
        metrics_dict['Total']['tp'] += tp
        metrics_dict['Total']['fp'] += fp
        metrics_dict['Total']['tn'] += tn
        metrics_dict['Total']['fn'] += fn
    
    return metrics_dict

def calculate_agg_metrics(tp, fp, tn, fn):
    # Calculate metrics
    accuracy = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    metrics = {'Accuracy': accuracy, 'F1 Score': f1, 'Precision': precision, 'Recall': recall}
    return metrics

def calculate_positioning_error(frame_results_df, threshold=4):
    positioning_error = []
    for i in range(len(frame_results_df)):
        row = frame_results_df.iloc[i]
        predicted_position = row['predicted_position']
        actual_position = row['ground_truth_position']

        if actual_position != (-1,-1) and predicted_position != (-1,-1):
            error = distance.euclidean(predicted_position, actual_position)
            positioning_error.append(error)

    # get mean PE
    mean_positioning_error = np.mean(positioning_error)

    # get mean PE below TH
    mean_positioning_error_below_th = np.mean([pe for pe in positioning_error if pe < threshold])

    # count bins for 0, 1,2,3,4,1000
    bins = [0, 1, 2, 3, 4, 5, 10, 100000]

    # create counts
    counts = np.bincount(np.digitize(positioning_error, bins))
    
    return counts, mean_positioning_error, mean_positioning_error_below_th

## Calculate Metrics

In [5]:
# loop over rows and add f1 metrics
results['Positioning_Error_Counts'] = None  # Initialize the column
results['Positioning_Error_Counts'] = results['Positioning_Error_Counts'].astype(object)
for i in range(len(results)):
    frame_results = results.iloc[i]['frame_results']

    frame_results_df = pd.DataFrame(eval(frame_results))

    # calculate metrics
    metrics_dict = calculate_df_metrcis(frame_results_df)

    # calculate aggregated metrics
    agg_metrics = {}
    for key, value in metrics_dict.items():
        tp = value['tp']
        fp = value['fp']
        tn = value['tn']
        fn = value['fn']
        
        agg_metrics[key] = calculate_agg_metrics(tp, fp, tn, fn)

    # filter for only f1-score
    agg_metrics_f1 = {key: value['F1 Score'] for key, value in agg_metrics.items()}

    # convert to single columns
    for key, value in agg_metrics_f1.items():
        results.at[i, key] = value

    # calculate positioning error
    counts, mean_positioning_error, mean_positioning_error_below_th = calculate_positioning_error(frame_results_df)

    results.at[i, 'Positioning_Error_mean'] = mean_positioning_error
    results.at[i, 'Positioning_Error_mean_Below_Threshold'] = mean_positioning_error_below_th

    #add counts as a an object to the row
    results.at[i, 'Positioning_Error_Counts'] = counts.tolist()


# remove column frame_results
results = results.drop(columns=['frame_results'])

In [6]:
# Convert the 'version' column from string representation of dictionaries to actual dictionaries
results['version'] = results['version'].apply(eval)  # Use eval to convert strings to dictionaries

# Expand the dictionary into separate columns
version_df = pd.json_normalize(results['version'])

# Drop the original 'version' column and concatenate the expanded columns
results = pd.concat([results.drop(columns=['version']), version_df], axis=1)


In [7]:
# create a new version column that has a concat string
results['Model'] = "TrackNet " + results['input_number'].astype(str) + "-in-"+results['output_number'].astype(str) + "-out aug-"+results['augmentation'].astype(str)
#put version column in front
results = results[['Model'] + [col for col in results.columns if col != 'Model']]

In [8]:
# rename and filter columns
rename_dict = {
    'Model': 'Model',
    'f1': 'F1-Score Total',
    'precision': 'Precision Total',
    'recall': 'Recall Total',
    'Dataset_New': 'F1 Dataset New',
    'Dataset_TrackNet': 'F1 Dataset TrackNet',
    'Visibility_Visible': 'F1 Visibility Visibile',
    'Visibility_Difficult': 'F1 Visibility Difficult',
    'Visibility_Occluded': 'F1 Visibility Occluded',
    'Visibility_Blurry': 'F1 Visibility Blurry',
    'fps': 'FPS',
    'Positioning_Error_mean': 'PE Mean',
    'Positioning_Error_mean_Below_Threshold': 'PE Mean Below Threshold',
    'Positioning_Error_Counts': 'PE Counts',
}

results = results.rename(columns=rename_dict)

# filter columns
results = results[['Model', 'F1-Score Total', 'Precision Total', 'Recall Total', 'F1 Dataset New', 'F1 Dataset TrackNet', 'F1 Visibility Visibile', 'F1 Visibility Difficult', 'F1 Visibility Occluded', 'F1 Visibility Blurry', 'FPS', 'PE Mean', 'PE Mean Below Threshold', 'PE Counts']]
results

,Model,F1-Score Total,Precision Total,Recall Total,F1 Dataset New,F1 Dataset TrackNet,F1 Visibility Visibile,F1 Visibility Difficult,F1 Visibility Occluded,F1 Visibility Blurry,FPS,PE Mean,PE Mean Below Threshold,PE Counts
0,TrackNet 3-in-3-out aug-True,0.921673,0.942683,0.901578,0.900658,0.960961,0.962884,0.769441,0.755556,0.990826,330.915950,3.042070,0.684447,"[0, 975, 1063, 110, 21, 11, 15, 60]"
1,TrackNet 3-in-3-out aug-False,0.907603,0.923313,0.892418,0.878278,0.960914,0.954849,0.722380,0.727273,0.921569,324.674231,3.087418,0.657174,"[0, 980, 1010, 93, 22, 9, 33, 91]"
2,TrackNet 3-in-1-out aug-True,0.913890,0.946661,0.883312,0.891494,0.954768,0.955407,0.734923,0.600000,0.942308,135.161485,3.351784,0.662389,"[0, 985, 1014, 80, 33, 13, 14, 72]"
3,TrackNet 3-in-1-out aug-False,0.878928,0.901713,0.857265,0.854527,0.923845,0.932897,0.685131,0.634146,0.865979,136.058206,2.883454,0.629702,"[0, 948, 959, 73, 20, 16, 32, 109]"
4,TrackNet 5-in-3-out aug-True,0.894102,0.905800,0.882703,0.867462,0.944513,0.946121,0.727786,0.634146,0.932039,311.834388,5.434755,0.798676,"[0, 776, 1127, 131, 41, 26, 20, 95]"
5,TrackNet 5-in-3-out aug-False,0.917113,0.946089,0.889859,0.896712,0.955206,0.961232,0.736695,0.697674,0.962264,304.730405,2.007926,0.654637,"[0, 989, 1039, 91, 20, 16, 10, 62]"
6,TrackNet 5-in-5-out aug-True,0.926477,0.957400,0.897489,0.913855,0.954016,0.959021,0.797333,0.808511,0.981481,448.019195,3.420916,0.693266,"[0, 949, 1098, 99, 27, 8, 12, 56]"
7,TrackNet 5-in-5-out aug-False,0.915556,0.928326,0.903132,0.893198,0.960241,0.960956,0.764384,0.727273,0.971963,419.539229,2.079026,0.705929,"[0, 940, 1066, 124, 26, 16, 34, 60]"


In [ ]:
results.to_csv('../results/Results_TN.csv', index=False)